# Manage spatial data

In [ ]:
!lamin init --storage ./test-spatial --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import matplotlib.pyplot as plt
import scanpy as sc

lb.settings.species = "human"
ln.settings.verbosity = 3

In [ ]:
ln.track()

Here we have a spatial gene expression dataset measured using [Visium](https://www.10xgenomics.com/products/spatial-gene-expression) from [Suo22](https://developmental.cellatlas.io/fetal-immune).

This dataset contains two parts:

1. a high-res image of a slice of fetal liver
2. a single cell expression dataset in .h5ad

In [ ]:
img_path = ln.dev.datasets.file_tiff_suo22()
img = plt.imread(img_path)
plt.imshow(img)
plt.show()

In [ ]:
adata = ln.dev.datasets.anndata_suo22_Visium10X()
# subset to the same image
adata = adata[adata.obs["img_id"] == "F121_LP1_4LIV"].copy()
adata

In [ ]:
# plot where CD45+ leukocytes are in the slice
sc.pl.scatter(adata, "array_row", "array_col", color="ENSG00000081237")

## Register the AnnData and image file as a dataset

In [ ]:
file_ad = ln.File.from_anndata(
    adata,
    description="Suo22 Visium10X image F121_LP1_4LIV",
    var_ref=lb.Gene.ensembl_gene_id,
)
file_ad.save()

In [ ]:
file_img = ln.File(img_path, description="Suo22 image F121_LP1_4LIV")
file_img.save()

In [ ]:
dataset = ln.Dataset.from_files(files=[file_ad, file_img], name="Suo22")
dataset.save()

In [ ]:
# clean up test instance
!lamin delete test-spatial
!rm -r test-flow